In [1]:
# imports
import os, rasterio
import time
from deepforest import main, utilities, get_data, preprocess 
from os.path import exists
from  PIL import Image, ImagePath
Image.MAX_IMAGE_PIXELS = None
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
image_path = get_data("D:\\Dropbox\\P.Robeck\\BPLA Dropbox\\07 Temp\\Hankos_Submittals\\221001_DQ_ECW2022\\img\\DQ_2022_3b.tif")
print(exists(image_path))

# annotations = ("D:\\Dropbox\\P.Robeck\\BPLA Dropbox\\07 Temp\\Hankos_Submittals\\221001_DQ_ECW2022\\annotation\\deep-forest-training.csv")
# print(exists(annotations))

exImg=Image.open(image_path)
exImg.getbands()

True


('R', 'G', 'B')

In [ ]:
ano_shp="D:\\Dropbox\\P.Robeck\\BPLA Dropbox\\07 Temp\Hankos_Submittals\\221001_DQ_ECW2022\\annotation\\deep-forest-training.shp"
annotations=utilities.shapefile_to_annotations(ano_shp, image_path,  savedir='.')



In [ ]:
annotations.to_csv("D:\\Dropbox\\P.Robeck\\BPLA Dropbox\\07 Temp\Hankos_Submittals\\221001_DQ_ECW2022\\annotation\\deep-forest-training-2.csv")

In [ ]:
ano_csv=r"D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\crop\DQ_2022_3b.csv"

In [ ]:
#Write csv to file and crop
tmpdir = ("D:\\Dropbox\\P.Robeck\\BPLA Dropbox\\07 Temp\\Hankos_Submittals\\221001_DQ_ECW2022\\crop\\")
# annotations.to_csv("{}/example.csv".format(tmpdir), index=False)
annotations_file = preprocess.split_raster(path_to_raster=image_path,
                                           annotations_file=ano_csv,
                                           base_dir=tmpdir,
                                           patch_size=500,
                                           patch_overlap=0.25)

# Returns a 6 column pandas array
# 

In [ ]:
del(annotations_file['Unnamed: 0'])
assert annotations_file.shape[1] == 6
annotations_file


In [3]:
ano_split=get_data(r"D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\crop\DQ_2022_3b.csv")
exists(ano_split)

True

In [8]:
ano_split
# os.path.dirname(ano_split)

'D:\\Dropbox\\P.Robeck\\BPLA Dropbox\\07 Temp\\Hankos_Submittals\\221001_DQ_ECW2022\\crop\\DQ_2022_3b.csv'

In [4]:
model = main.deepforest()

# Example run with short training
model.config["train"]['epochs'] = 5
# model.config['gpus'] = '-1' #move to GPU and use all the GPU resources
# model.config['cpu'] = 'accelerator'
model.config["save-snapshot"] = False
model.config["train"]["csv_file"] = ano_split
model.config["train"]["root_dir"] = os.path.dirname(ano_split)
model.config['num_worker']=16
# model.config['max_epochs']=10
model.create_trainer()
model.use_release()


Reading config file: c:\Users\Robeck\anaconda3\envs\df-tree\lib\site-packages\deepforest\data\deepforest_config.yml


c:\Users\Robeck\anaconda3\envs\df-tree\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Robeck\anaconda3\envs\df-tree\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RetinaNet_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=RetinaNet_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Model from DeepForest release https://github.com/weecology/DeepForest/releases/tag/1.0.0 was already downloaded. Loading model from file.
Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/1.0.0


In [5]:
start_time = time.time()
model.trainer.fit(model)
print(f"--- Training on CPU: {(time.time() - start_time):.2f} seconds ---")


  | Name  | Type      | Params
------------------------------------
0 | model | RetinaNet | 32.1 M
------------------------------------
31.9 M    Trainable params
222 K     Non-trainable params
32.1 M    Total params
128.592   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\Robeck\anaconda3\envs\df-tree\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:366: UserWarning: One of given dataloaders is None and it will be skipped.
  rank_zero_warn("One of given dataloaders is None and it will be skipped.")
c:\Users\Robeck\anaconda3\envs\df-tree\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


--- Training on CPU: 1214.79 seconds ---


In [6]:
# boxes = model.predict_image(path=image_path, return_plot = True)
predicted_raster = model.predict_tile(image_path, return_plot = False, patch_size=500,patch_overlap=0.25)


100%|██████████| 1302/1302 [24:26<00:00,  1.13s/it]

No predictions made, returning None
No predictions made, returning None


In [7]:
pd.DataFrame(predicted_raster)

""


In [ ]:
r = rasterio.open(image_path)
transform = r.transform 
crs = r.crs
print(crs)

gdf = utilities.annotations_to_shapefile(predicted_raster, transform=transform, crs=crs)

In [ ]:
gdf.to_file('trained-500-1e.shp')

In [ ]:
# Importing Image class from PIL module
 
# Opens a image in RGB mode
RUH_img = Image.open(r"\\?\D:\Dropbox\P.Robeck\BPLA Dropbox\08 GIS-data\GIS-data-Riyadh\2022-03-world-view-3\RCRC-4Bands-Mosaic_FIN_I542628_614813\RCRC-4Bands-Mosaic_FIN_I542628_614813\RCRC-4Bands-Mosaic_I542628_FL01-P244705\RCRC-4Bands-Mosaic_MOS\22FEB15073634-S2AM-RCRC-4Bands-Mosaic.TIF")

DQ_img = Image.open(r"D:\\Dropbox\\P.Robeck\\BPLA Dropbox\\07 Temp\\Hankos_Submittals\\221001_DQ_ECW2022\\PNG\\DQ_2022_3b.png")


In [ ]:
DQ_img.getbbox()

In [ ]:
print(DQ_img.format, DQ_img.size, DQ_img.mode,DQ_img.getbbox)